<a href="https://colab.research.google.com/github/Praveenku32k/Deep-Learning-Implementation/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
ROOT="/content/drive/MyDrive/Artifical_Neural_Network_class_Lab"

In [2]:
import os
os.chdir(ROOT)

In [3]:
assert os.getcwd() ==ROOT

In [4]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [5]:
(X_train_full,y_train_full),(X_test,y_test)=tf.keras.datasets.mnist.load_data()
X_train_full=X_train_full /255.0
X_test=X_test /255.0
X_valid,X_train=X_train_full[:5000],X_train_full[5000:]
y_valid,y_train=y_train_full[:5000],y_train_full[5000:]

11501568/11490434 [==============================] - 0s 0us/step


In [6]:
tf.random.set_seed(42)#I AM USING SET_SEED
np.random.seed(42)

LAYERS=[
        tf.keras.layers.Flatten(input_shape=[28,28]),#WHY WE ARE USING he_normal
        tf.keras.layers.Dense(300,kernel_initializer="he_normal"),#HE_NORAMAL IS VARIENT OF ReLU.#kernal initilizer ,itnothing bbut the weight update rule.
        tf.keras.layers.LeakyReLU(),#Here i am using LeakyRelu function 
        tf.keras.layers.Dense(100,kernel_initializer="he_normal"),
        tf.keras.layers.LeakyReLU(),
        tf.keras.layers.Dense(10,activation="softmax")

]
model=tf.keras.models.Sequential(LAYERS)

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"]
              )

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [9]:
histroy=model.fit(X_train,y_train,epochs=10,
                  validation_data=(X_valid,y_valid),verbose=2)

Epoch 1/10
1719/1719 - 7s - loss: 1.5275 - accuracy: 0.5970 - val_loss: 0.9444 - val_accuracy: 0.7980
Epoch 2/10
1719/1719 - 4s - loss: 0.7465 - accuracy: 0.8287 - val_loss: 0.5868 - val_accuracy: 0.8596
Epoch 3/10
1719/1719 - 4s - loss: 0.5412 - accuracy: 0.8624 - val_loss: 0.4685 - val_accuracy: 0.8834
Epoch 4/10
1719/1719 - 4s - loss: 0.4591 - accuracy: 0.8771 - val_loss: 0.4104 - val_accuracy: 0.8940
Epoch 5/10
1719/1719 - 4s - loss: 0.4142 - accuracy: 0.8869 - val_loss: 0.3758 - val_accuracy: 0.9006
Epoch 6/10
1719/1719 - 4s - loss: 0.3852 - accuracy: 0.8938 - val_loss: 0.3525 - val_accuracy: 0.9052
Epoch 7/10
1719/1719 - 4s - loss: 0.3644 - accuracy: 0.8979 - val_loss: 0.3348 - val_accuracy: 0.9102
Epoch 8/10
1719/1719 - 4s - loss: 0.3485 - accuracy: 0.9022 - val_loss: 0.3209 - val_accuracy: 0.9138
Epoch 9/10
1719/1719 - 4s - loss: 0.3356 - accuracy: 0.9053 - val_loss: 0.3111 - val_accuracy: 0.9152
Epoch 10/10
1719/1719 - 4s - loss: 0.3251 - accuracy: 0.9077 - val_loss: 0.3016 - 

In [10]:
model.save("Pretrained_mnist_model.h5")

###Transer Leraning

In [11]:
Pretrained_mnist_model=tf.keras.models.load_model("Pretrained_mnist_model.h5")

In [12]:
Pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                1010      
Total params: 266,610
Trainable params: 266,610
Non-trainable params: 0
__________________________________________________

In [13]:
for layer in Pretrained_mnist_model.layers:
  print(f"{layer.name}:{layer.trainable}")

flatten:True
dense:True
leaky_re_lu:True
dense_1:True
leaky_re_lu_1:True
dense_2:True


In [14]:
for layer in Pretrained_mnist_model.layers[:-1]:#Here we making rthe last row as flase so we are doing [:-1]
  layer.trainable=False
  print(f"{layer.name}:{layer.trainable}")

flatten:False
dense:False
leaky_re_lu:False
dense_1:False
leaky_re_lu_1:False


In [15]:
for layer in Pretrained_mnist_model.layers:
  print(f"{layer.name}:{layer.trainable}")#only last is trained as false  we can see it 

flatten:False
dense:False
leaky_re_lu:False
dense_1:False
leaky_re_lu_1:False
dense_2:True


In [16]:
lower_Pretrained_layers=Pretrained_mnist_model.layers[:-1]
new_model=tf.keras.models.Sequential(lower_Pretrained_layers)
new_model.add(
    tf.keras.layers.Dense(2,activation="softmax")
)

In [17]:
new_model.summary()#Non-trainable params: 265,600 and here we are training the last row only .

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 300)               235500    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               30100     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 202       
Total params: 265,802
Trainable params: 202
Non-trainable params: 265,600
______________________________________________

*** *ONLY I AM TRAINING THE LAST ROW IS GOING TO BE TRAINED  NOT THE OTHERS ROWS* ***

In [18]:
np.unique(y_train)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [19]:
#y_train_bin,y_test_bin,y_valid_bin             #find even odd value from the y_train

In [20]:
np.where(y_train%2==0,1,0)

array([0, 0, 1, ..., 0, 1, 1])

In [21]:
y_train[0], y_train[-1]#checking for odd even

(7, 8)

In [22]:
def update_even_odd_labels(labels):
  for idx,label in enumerate(labels):
    labels[idx]=np.where(label %2==0,1,0)
  return labels

In [23]:
y_train_bin,y_test_bin,y_valid_bin=update_even_odd_labels([y_train,y_test,y_valid])

In [24]:
np.unique(y_valid_bin)

array([0, 1])

In [25]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"]
              )
#here is accuarcy is more then the previous trained .example of child and a graguate student who is more trained.
#model zoo for in tensorflow .you can utilize this model.

histroy=new_model.fit(X_train,y_train_bin,epochs=10,
                  validation_data=(X_valid,y_valid_bin),verbose=2)

Epoch 1/10
1719/1719 - 4s - loss: 0.3898 - accuracy: 0.8288 - val_loss: 0.3247 - val_accuracy: 0.8676
Epoch 2/10
1719/1719 - 4s - loss: 0.3300 - accuracy: 0.8602 - val_loss: 0.3049 - val_accuracy: 0.8754
Epoch 3/10
1719/1719 - 4s - loss: 0.3163 - accuracy: 0.8660 - val_loss: 0.2948 - val_accuracy: 0.8796
Epoch 4/10
1719/1719 - 4s - loss: 0.3083 - accuracy: 0.8701 - val_loss: 0.2884 - val_accuracy: 0.8832
Epoch 5/10
1719/1719 - 4s - loss: 0.3023 - accuracy: 0.8725 - val_loss: 0.2834 - val_accuracy: 0.8846
Epoch 6/10
1719/1719 - 4s - loss: 0.2978 - accuracy: 0.8753 - val_loss: 0.2792 - val_accuracy: 0.8874
Epoch 7/10
1719/1719 - 4s - loss: 0.2939 - accuracy: 0.8772 - val_loss: 0.2758 - val_accuracy: 0.8872
Epoch 8/10
1719/1719 - 4s - loss: 0.2907 - accuracy: 0.8788 - val_loss: 0.2728 - val_accuracy: 0.8890
Epoch 9/10
1719/1719 - 4s - loss: 0.2876 - accuracy: 0.8797 - val_loss: 0.2708 - val_accuracy: 0.8906
Epoch 10/10
1719/1719 - 4s - loss: 0.2851 - accuracy: 0.8817 - val_loss: 0.2678 - 

In [26]:
new_model.evaluate(X_test,y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.2768 - accuracy: 0.8861


[0.2768370509147644, 0.8860999941825867]

In [27]:
X_new=X_test[:3]

y_test[:3],y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([0, 1, 0]))

In [28]:
np.argmax(new_model.predict(X_new),axis=-1)

array([0, 1, 0])

In [29]:
def update_even_odd_labels(labels):#CHECK IT IS GREATER THEN 5 THE IT RETURN 1 ELSE IT RETRUN 0
  for idx,label in enumerate(labels):
    labels[idx]=np.where(label>5,1,0)#Here i am doing the changing 
  return labels

In [30]:
y_train_bin,y_test_bin,y_valid_bin=update_even_odd_labels([y_train,y_test,y_valid])

In [31]:
np.unique(y_valid_bin)

array([0, 1])

In [32]:
new_model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"]
              )
#here is accuarcy is more then the previous trained .example of child and a graguate student who is more trained.
#model zoo for in tensorflow .you can utilize this model.

histroy=new_model.fit(X_train,y_train_bin,epochs=10,
                  validation_data=(X_valid,y_valid_bin),verbose=2)

Epoch 1/10
1719/1719 - 4s - loss: 0.6059 - accuracy: 0.7084 - val_loss: 0.4501 - val_accuracy: 0.7822
Epoch 2/10
1719/1719 - 4s - loss: 0.4172 - accuracy: 0.8162 - val_loss: 0.3871 - val_accuracy: 0.8392
Epoch 3/10
1719/1719 - 3s - loss: 0.3813 - accuracy: 0.8428 - val_loss: 0.3611 - val_accuracy: 0.8556
Epoch 4/10
1719/1719 - 4s - loss: 0.3635 - accuracy: 0.8536 - val_loss: 0.3471 - val_accuracy: 0.8628
Epoch 5/10
1719/1719 - 4s - loss: 0.3520 - accuracy: 0.8596 - val_loss: 0.3352 - val_accuracy: 0.8642
Epoch 6/10
1719/1719 - 4s - loss: 0.3435 - accuracy: 0.8638 - val_loss: 0.3279 - val_accuracy: 0.8674
Epoch 7/10
1719/1719 - 4s - loss: 0.3370 - accuracy: 0.8663 - val_loss: 0.3208 - val_accuracy: 0.8730
Epoch 8/10
1719/1719 - 4s - loss: 0.3315 - accuracy: 0.8697 - val_loss: 0.3159 - val_accuracy: 0.8746
Epoch 9/10
1719/1719 - 4s - loss: 0.3270 - accuracy: 0.8716 - val_loss: 0.3112 - val_accuracy: 0.8776
Epoch 10/10
1719/1719 - 4s - loss: 0.3232 - accuracy: 0.8733 - val_loss: 0.3069 - 

In [33]:
new_model.evaluate(X_test,y_test_bin)

313/313 [==============================] - 1s 2ms/step - loss: 0.3133 - accuracy: 0.8789


[0.3132910430431366, 0.8788999915122986]

In [34]:
X_new=X_test[:3]

y_test[:3],y_test_bin[:3]

(array([7, 2, 1], dtype=uint8), array([1, 0, 0]))

In [35]:
np.argmax(new_model.predict(X_new),axis=-1)

array([1, 0, 0])